# 残差网络（Residual Networks）

在本节中，你将学习如何构建非常深的卷积神经网络，使用残差网络（ResNets）。理论上，非常深的网络可以表示非常复杂的函数；但在实践中，它们很难训练。由 [He 等人](https://arxiv.org/pdf/1512.03385.pdf) 提出的残差网络，使你可以训练比以往更深的网络。

**在本节中，你将：**
- 实现 ResNets 的基本构建模块。
- 将这些模块组合起来，实现并训练一个用于图像分类的先进神经网络。

在开始之前，先运行下面的代码以加载所需的包。


### 1 导入库与环境设置

In [2]:
# ==============================
# 导入必要的库
# ==============================

# ------------------------------
# 数值计算
# ------------------------------
import numpy as np                   # NumPy，用于高效的数组和矩阵运算

# ------------------------------
# PyTorch 核心模块
# ------------------------------
import torch                         # PyTorch 主模块
import torch.nn as nn                # 神经网络模块（定义模型用）
import torch.nn.functional as F      # 常用函数模块（激活函数、卷积操作等）
import torch.optim as optim          # 优化器模块（如 SGD, Adam）

# ------------------------------
# 数据处理与模型相关
# ------------------------------
from torch.utils.data import Dataset, DataLoader, TensorDataset  
# Dataset: 自定义数据集类的基类
# DataLoader: 用于批量加载数据，支持 shuffle、batch 等
# TensorDataset: 可以直接把 Tensor 包装成 Dataset

from torchvision import datasets, transforms, models
# datasets: PyTorch 提供的常用数据集（MNIST、CIFAR 等）
# transforms: 图像预处理、数据增强
# models: torchvision 提供的预训练模型（ResNet, VGG 等）

# ------------------------------
# 图像处理与可视化
# ------------------------------
from PIL import Image               # PIL 图像处理库
import matplotlib.pyplot as plt     # 绘图库，用于显示图像、绘制曲线

# ------------------------------
# 系统与文件操作
# ------------------------------
import os                           # 操作系统接口（文件路径、目录创建等）

# ------------------------------
# HDF5 数据读取
# ------------------------------
import h5py                         # h5py：用于读取和写入 HDF5 文件，常用于存储大规模数据集

# ------------------------------
# 设置随机数种子
# ------------------------------
torch.manual_seed(42)               # PyTorch 随机数种子，保证可复现
np.random.seed(42)                  # NumPy 随机数种子，保证可复现


### 2 数据加载相关

让我们加载 SIGNS 数据集。

<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> **Figure 6** </u><font color='purple'>  : **SIGNS 数据集** </center></caption>

In [3]:
# ==============================
# 数据加载函数
# ==============================
def load_dataset():
    """
    功能：
        从 HDF5 文件中加载训练集和测试集的数据和标签
        
    返回：
        train_set_x_orig -- 训练集特征，shape = (m_train, 64, 64, 3)，每张图片为 64x64 RGB 图像
        train_set_y_orig -- 训练集标签，shape = (m_train,)，整数表示类别
        test_set_x_orig  -- 测试集特征，shape = (m_test, 64, 64, 3)
        test_set_y_orig  -- 测试集标签，shape = (m_test,)
        classes          -- 标签类别数组，shape = (n_classes,)
    """

    # ------------------------------
    # 加载训练集 HDF5 文件
    # ------------------------------
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    # train_dataset: HDF5 文件对象
    # 参数 "datasets/train_signs.h5" 表示文件路径
    # "r" 表示只读模式，不能修改文件内容

    train_set_x_orig = np.array(train_dataset["train_set_x"][:])
    # train_dataset["train_set_x"]: HDF5 文件中的训练特征数据集
    # [:] 表示读取整个数据集
    # np.array(...) 将 HDF5 数据转为 numpy 数组，方便后续处理
    # 结果 shape = (m_train, 64, 64, 3)

    train_set_y_orig = np.array(train_dataset["train_set_y"][:])
    # train_dataset["train_set_y"]: HDF5 文件中的训练标签数据集
    # np.array(...) 转为 numpy 数组
    # 结果 shape = (m_train,)，每个元素为整数类别

    # ------------------------------
    # 加载测试集 HDF5 文件
    # ------------------------------
    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    # test_dataset: HDF5 文件对象，测试集数据
    # "r" 表示只读模式

    test_set_x_orig = np.array(test_dataset["test_set_x"][:])
    # 测试集特征数据，shape = (m_test, 64, 64, 3)

    test_set_y_orig = np.array(test_dataset["test_set_y"][:])
    # 测试集标签数据，shape = (m_test,)

    classes = np.array(test_dataset["list_classes"][:])
    # list_classes: 类别列表，如 [0,1,2,...,5]
    # np.array(...) 转为 numpy 数组，方便索引和分类

    # ------------------------------
    # 返回加载的数据
    # ------------------------------
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [4]:
# 调用之前定义的 load_dataset() 函数，加载训练集和测试集
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
# X_train_orig: 训练集特征，shape = (m_train, 64, 64, 3)
# Y_train_orig: 训练集标签，shape = (m_train,)
# X_test_orig: 测试集特征，shape = (m_test, 64, 64, 3)
# Y_test_orig: 测试集标签，shape = (m_test,)
# classes: 标签类别数组

In [5]:
# ==============================
# 自定义 Dataset类
# ==============================
class SignsDataset(Dataset):
    """
    功能：
        将 NumPy 数据集封装为 PyTorch Dataset，用于 DataLoader 迭代
        支持图像归一化、通道转换以及可选 transform
    """

    def __init__(self, X_np, Y_np, transform=None):
        """
        初始化 Dataset

        参数：
        X_np     -- NumPy 图像数组，shape = (m,64,64,3)，dtype uint8 或 float32
        Y_np     -- NumPy 标签数组，shape = (m,) 或 (m,1)
        transform-- 可选图像变换（如 torchvision.transforms）
        """

        # ------------------------------
        # 特征数据转换为 float32 类型
        # ------------------------------
        self.X = X_np.astype(np.float32)
        # astype(np.float32)：确保数据类型为 float32，以便后续 PyTorch 计算

        # 如果最大值 > 2.0，说明是原始 0-255 图像，需要归一化到 [0,1]
        if self.X.max() > 2.0:
            self.X = self.X / 255.0

        # ------------------------------
        # PyTorch 要求图像通道在前（C,H,W）
        # ------------------------------
        self.X = np.transpose(self.X, (0,3,1,2))
        # np.transpose：调整数组维度顺序
        # (0,3,1,2) 说明：
        # 0: 样本数 m 不变
        # 3: 通道数 C 从最后一维移到第二维
        # 1: 高度 H
        # 2: 宽度 W
        # 最终 shape = (m,3,64,64)

        # ------------------------------
        # 标签处理
        # ------------------------------
        self.y = np.array(Y_np).reshape(-1).astype(np.int64)
        # reshape(-1)：确保标签为一维向量
        # int64：PyTorch 分类任务要求标签为 long 类型

        # ------------------------------
        # 图像变换（可选）
        # ------------------------------
        self.transform = transform
        # 如果传入 transform（如随机裁剪、旋转等），后续 __getitem__ 会应用

    # ------------------------------
    # 返回样本总数
    # ------------------------------
    def __len__(self):
        return self.X.shape[0]
        # Dataset 长度就是样本数 m

    # ------------------------------
    # 返回单个样本和标签
    # ------------------------------
    def __getitem__(self, idx):
        """
        功能：
            根据索引 idx 返回单个样本图像和标签

        参数：
        idx -- 样本索引，整数

        返回：
        img   -- torch.Tensor 图像，shape = (3,64,64)，dtype float32
        label -- torch.Tensor 标签，dtype long
        """

        # 取第 idx 张图像
        img = self.X[idx]       

        # 取对应标签
        label = self.y[idx]     

        # 如果定义了 transform，应用图像变换
        if self.transform:
            img = self.transform(img)

        # 转换为 PyTorch Tensor
        return torch.from_numpy(img), torch.tensor(label, dtype=torch.long)
    
        # ------------------------------
        # torch.from_numpy(img)
        # 功能：将 NumPy 数组 img 转换为 PyTorch Tensor
        # 输入 img：
        #   - 形状为 (C, H, W)，通道数在前
        #   - 数据类型通常为 float32
        # 输出：
        #   - PyTorch Tensor，保持原数组数据
        #   - 与原 NumPy 数组共享内存，修改 Tensor 会影响原数组
        # 用途：
        #   - PyTorch 模型训练需要 Tensor 输入，而不是 NumPy 数组
        #   - 这里 img 是单张图像，用于 Dataset 返回一个样本
        
        # torch.tensor(label, dtype=torch.long)
        # 功能：将单个标签 label 转换为 PyTorch Tensor
        # 参数：
        #   - label：样本的类别标签（整数）
        #   - dtype=torch.long：指定数据类型为 long (int64)，PyTorch 分类任务要求
        # 输出：
        #   - PyTorch Tensor，形状为标量 ( ) 或 (1,)
        #   - 数据类型为 torch.int64
        # 用途：
        #   - 作为目标标签，输入到分类损失函数 nn.CrossEntropyLoss 时必须是 long 类型


In [6]:
# ==============================
# 创建 Dataset 和 DataLoader
# ==============================

# 定义类别数
num_classes = 6
# 原始脚本中 X_train/X_test 已除以 255，这里在 SignsDataset 内部已做归一化

# 使用自定义 SignsDataset 封装训练集和测试集
train_dataset = SignsDataset(X_train_orig, Y_train_orig)
# train_dataset: 封装训练集的 Dataset 对象，可用于 DataLoader 迭代
test_dataset  = SignsDataset(X_test_orig,  Y_test_orig)
# test_dataset: 封装测试集的 Dataset 对象

# 定义每个 batch 的大小
batch_size = 64

# ------------------------------
# 设置随机种子保证 DataLoader shuffle 可复现
# ------------------------------
g = torch.Generator()   # 创建一个随机数生成器
g.manual_seed(3)        # 设置固定种子 3，确保每次 shuffle 顺序相同

# 创建训练集 DataLoader
train_loader = DataLoader(
    train_dataset,      # 数据集对象
    batch_size=batch_size,  # 每个 batch 的样本数
    shuffle=True,       # 打乱样本顺序
    generator=g         # 使用固定种子生成器，保证可复现
)

# 创建测试集 DataLoader
test_loader  = DataLoader(
    test_dataset,       # 数据集对象
    batch_size=batch_size,  # 每个 batch 的样本数
    shuffle=False       # 测试集不打乱
)



## 3 - 非常深的神经网络问题
近年来，神经网络越来越深，最先进的网络从最初的几层（例如 AlexNet）发展到上百层甚至更多。

### 深度网络的优势
非常深的网络主要优势在于：
1. **表示能力强**：可以拟合非常复杂的函数。
2. **特征学习层次丰富**：  
   - 较浅的层学习低级特征，例如边缘（edges）。  
   - 较深的层学习高级特征，例如对象的复杂结构或纹理。  

### 深度网络的训练难点
尽管深层网络能力强，但**更深的网络并不总是效果更好**。主要障碍是 **梯度消失（vanishing gradients）**：

- 在深层网络中，梯度信号在向前传播和反向传播时可能迅速衰减为零。
- 在梯度下降训练过程中，当你从最后一层反向传播到第一层时，每一步都会与权重矩阵相乘，因此梯度可能指数级衰减（vanishing）或在少数情况下指数级增长（exploding）。

### 梯度消失的表现
- 在训练过程中，你可能会观察到**前几层梯度的大小（或范数）迅速变为零**，导致梯度下降几乎无法更新这些层的权重。
- 这会使得训练非常缓慢甚至陷入停滞。

> 小结：非常深的神经网络可以学习复杂的特征，但梯度消失问题会阻碍前几层权重的有效更新，需要特殊的网络结构或训练技巧来解决。


<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> 
<u> <font color='purple'> **图 1** </u><font color='purple'> ：**梯度消失示意图** <br> 
随着网络训练，前几层的学习速度会非常迅速地下降
</center></caption>

---

### 解决梯度消失问题：残差网络（Residual Network, ResNet）

- 如图 1 所示，普通深度神经网络在训练时，前几层的梯度会迅速减小，导致学习变慢。
- 为了解决这个问题，我们可以使用 **残差网络（ResNet）**：
  1. ResNet 通过 **残差模块（residual block）** 引入跳跃连接（skip connections）。
  2. 这些跳跃连接允许梯度直接沿着网络向前或向后传播，**缓解梯度消失**。
  3. 这样，即使网络非常深，也能保持前几层的有效学习。
  
> 小结：接下来，我们将通过构建 ResNet，学习如何训练非常深的卷积神经网络，同时避免梯度消失的问题。


## 4 - 构建残差网络（Residual Network, ResNet）

在 ResNet 中，**“捷径（shortcut）”或“跳跃连接（skip connection）”** 允许梯度直接反向传播到前面的层，从而缓解梯度消失问题：  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> 
<u> <font color='purple'> **图 2** </u><font color='purple'> ：一个 ResNet 模块示意图，展示了 **跳跃连接** <br> 
</center></caption>

---

### 图示解读
- 左侧图：表示网络的“主路径”（main path）。
- 右侧图：在主路径上添加了一个跳跃连接（shortcut）。
- 通过将这样的 ResNet 模块堆叠起来，就可以形成一个**非常深的网络**。

---

### ResNet 模块的优势
1. **轻松学习恒等映射（identity function）**：
   - 如果某个模块只需要输出等于输入，跳跃连接可以让网络直接实现恒等映射。
   - 这意味着即使堆叠更多的模块，也不会损害训练集性能。
2. **缓解梯度消失**：
   - 跳跃连接允许梯度直


### 4.1 - 恒等块（Identity Block）

**恒等块**是 ResNet 中标准的模块，用于输入激活（$a^{[l]}$）和输出激活（$a^{[l+2]}$）**维度相同**的情况。  

#### 模块示意图

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> 
<u> <font color='purple'> **图 3** </u><font color='purple'> ：恒等块示意图。跳跃连接（shortcut）“跨越”2层。 
</center></caption>

- 上方路径是 **shortcut path（捷径路径）**  
- 下方路径是 **main path（主路径））**  
- 图中明确标出了每一层的 **CONV2D + ReLU** 步骤  
- 为了加快训练，还加入了 **BatchNorm（批量归一化）**  
- 在 PyTorch中，BatchNorm 只需一行代码即可实现，非常简单  

---

#### 升级版恒等块

在本练习中，你将实现一个**更强大的恒等块**，其跳跃连接跨越 **3 层隐藏层** 而非 2 层：

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> 
<u> <font color='purple'> **图 4** </u><font color='purple'> ：恒等块示意图。跳跃连接“跨越”3层。 
</center></caption>

---

#### 主路径（Main Path）的每一步

**第一部分**：
- CONV2D：$F_1$ 个卷积核，大小 (1,1)，步幅 (1,1)，padding="valid"，名称为 `conv_name_base + '2a'`  
- BatchNorm：沿通道维归一化，名称为 `bn_name_base + '2a'`  
- ReLU 激活函数：无名称、无超参数  

**第二部分**：
- CONV2D：$F_2$ 个卷积核，大小 (f,f)，步幅 (1,1)，padding="same"，名称为 `conv_name_base + '2b'`  
- BatchNorm：沿通道维归一化，名称为 `bn_name_base + '2b'`  
- ReLU 激活函数：无名称、无超参数  

**第三部分**：
- CONV2D：$F_3$ 个卷积核，大小 (1,1)，步幅 (1,1)，padding="valid"，名称为 `conv_name_base + '2c'`  
- BatchNorm：沿通道维归一化，名称为 `bn_name_base + '2c'`  
- **注意**：这一部分没有 ReLU 激活函数  

**最终步骤**：
- 将主路径输出与 **shortcut** 相加  
- 再应用 ReLU 激活函数  

---


> 小结：首先实现主路径的第一步（1x1 卷积 + BatchNorm + ReLU），然后依次完成第二步和第三步，最后与 shortcut 相加并激活即可。


In [7]:
# =====================================
# 恒等块 (Identity Block)
# =====================================
class IdentityBlock(nn.Module):
    """
    ResNet 恒等块：输入和输出通道数一致
    特点：
        - 输入 x 与输出 out 的维度相同
        - 通过三层卷积 + 批量归一化 + ReLU 激活
        - 最后通过残差连接（shortcut）实现加法
    """
    def __init__(self, in_channels, filters, f):
        """
        初始化恒等块

        参数：
            in_channels -- 输入通道数
            filters     -- list 类型 [F1, F2, F3]，三个卷积层的输出通道数
            f           -- 第二层卷积核大小 (中间卷积)
        """
        super(IdentityBlock, self).__init__()
        F1, F2, F3 = filters  # 解包三个卷积层的通道数

        # ------------------------------
        # 第一层卷积 1x1
        # ------------------------------
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,  # 输入通道
            out_channels=F1,          # 输出通道
            kernel_size=1,            # 卷积核 1x1
            stride=1,                 # 步长 1
            padding=0,                # 不填充
            bias=False                # 不使用偏置（因为后面有 BatchNorm）
        )
        self.bn1 = nn.BatchNorm2d(F1) # 批量归一化层

        # ------------------------------
        # 第二层卷积 fxf
        # ------------------------------
        self.conv2 = nn.Conv2d(
            in_channels=F1,           # 输入通道 = 第一层输出
            out_channels=F2,          # 输出通道
            kernel_size=f,            # 卷积核大小 fxf
            stride=1,                 # 步长 1
            padding=f//2,             # SAME 填充，保持宽高不变
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(F2)

        # ------------------------------
        # 第三层卷积 1x1
        # ------------------------------
        self.conv3 = nn.Conv2d(
            in_channels=F2, 
            out_channels=F3, 
            kernel_size=1, 
            stride=1, 
            padding=0, 
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(F3)

        # ReLU 激活函数
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        前向传播

        参数：
            x -- 输入 Tensor，形状 (batch_size, in_channels, H, W)

        返回：
            输出 Tensor，形状与输入相同 (batch_size, F3, H, W)
        """
        shortcut = x  # 保存残差（输入 x）

        # 第一层卷积 -> BN -> ReLU
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        # 第二层卷积 -> BN -> ReLU
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        # 第三层卷积 -> BN
        out = self.conv3(out)
        out = self.bn3(out)

        # 残差连接
        out += shortcut
        out = self.relu(out)

        return out


In [8]:
# =============================
# 测试恒等块 (Identity Block)
# =============================

# ------------------------------
# 设置随机种子，保证实验可复现
# ------------------------------
np.random.seed(1)
torch.manual_seed(1)

# ------------------------------
# 定义输入张量 X
# ------------------------------
# batch_size = 3, 通道数 = 6, 高 = 4, 宽 = 4
# 使用 NumPy 生成随机数，然后转为 PyTorch Tensor
X = torch.tensor(np.random.randn(3, 6, 4, 4), dtype=torch.float32)

# ------------------------------
# 定义恒等块
# ------------------------------
# 输入通道 = 6, 中间卷积核大小 f = 3, 三层卷积输出通道分别 = [2,4,6]
identity_block = IdentityBlock(in_channels=6, f=3, filters=[2,4,6])

# ------------------------------
# 分别在训练模式和评估模式下测试
# ------------------------------
for mode in ["train", "eval"]:
    if mode == "train":
        identity_block.train()  # 设置为训练模式（BN 会使用 batch 统计）
    else:
        identity_block.eval()   # 设置为评估模式（BN 使用滑动平均值）

    # 前向传播
    out = identity_block(X)

    # 打印输入输出统计信息
    print(f"\n模式: {mode}")
    print("输入: mean = %.4f, std = %.4f" % (X.mean().item(), X.std().item()))
    print("输出: mean = %.4f, std = %.4f" % (out.mean().item(), out.std().item()))
    # 展示输出前 6 个元素（展平后）
    print("out =", out.flatten()[:6].detach().numpy())



模式: train
输入: mean = 0.0810, std = 0.9571
输出: mean = 0.5569, std = 0.8006
out = [2.8718672  0.10689706 0.         0.         0.8642092  0.        ]

模式: eval
输入: mean = 0.0810, std = 0.9571
输出: mean = 0.4203, std = 0.5829
out = [1.6499243 0.        0.        0.        0.8540906 0.       ]


### 4.2 - 卷积块（Convolutional Block）

在实现了 ResNet 的恒等块（Identity Block）之后，另一种常用模块是 **卷积块（Convolutional Block）**。  
当输入和输出维度不一致时，就需要使用卷积块。它和恒等块的主要区别是：**shortcut（捷径路径）上也有一个 CONV2D 卷积层**。

#### 模块示意图

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> 
<u> <font color='purple'> **图 4** </u><font color='purple'> ：卷积块示意图 
</center></caption>

---

### 卷积块的作用

- shortcut 路径上的卷积层用于调整输入 $x$ 的维度，使主路径和捷径路径的输出维度一致，以便最终相加。
- 举例：如果希望将激活的高和宽缩小 2 倍，可以使用 1x1 卷积，stride=2。
- 这个卷积层不使用非线性激活函数，其主要作用是学习一个线性变换，使维度匹配，方便后续相加。

---

### 主路径（Main Path）每一步

**第一部分**：
- CONV2D：$F_1$ 个卷积核，大小 (1,1)，步幅 (s,s)，padding="valid"`
- BatchNorm：沿通道维归一化
- ReLU 激活函数：无名称、无超参数  

**第二部分**：
- CONV2D：$F_2$ 个卷积核，大小 (f,f)，步幅 (1,1)，padding="same"
- BatchNorm：沿通道维归一化
- ReLU 激活函数：无名称、无超参数  

**第三部分**：
- CONV2D：$F_3$ 个卷积核，大小 (1,1)，步幅 (1,1)，padding="valid"
- BatchNorm：沿通道维归一化  
- **注意**：这一部分没有 ReLU 激活函数  

---

### 捷径路径（Shortcut Path）

- CONV2D：$F_3$ 个卷积核，大小 (1,1)，步幅 (s,s)，padding="valid"`
- BatchNorm：沿通道维归一化，名称为   

---

### 最终步骤

- 将主路径输出与捷径路径输出相加  
- 再应用 ReLU 激活函数  

---


> 小结：卷积块和恒等块的差别主要在捷径路径上加了一个 1x1 卷积来调整维度，其余主路径逻辑与恒等块类似。


In [9]:
# =====================================
# 卷积块 (Convolutional Block)
# =====================================
class ConvolutionalBlock(nn.Module):
    """
    ResNet 卷积块：
        - 输入输出通道可能不一致
        - 需要通过卷积调整 shortcut
        - 主路径：Conv -> BN -> ReLU -> Conv -> BN -> ReLU -> Conv -> BN
        - 捷径路径：Conv1x1 + BN
        - 最后两者相加 -> ReLU
    """
    def __init__(self, in_channels, filters, f, s=2):
        """
        初始化卷积块

        参数：
            in_channels -- 输入通道数
            filters     -- list [F1, F2, F3]，主路径三层卷积输出通道数
            f           -- 第二层卷积核大小
            s           -- 主路径第一层卷积步幅，同时控制捷径卷积步幅
        """
        super(ConvolutionalBlock, self).__init__()
        F1, F2, F3 = filters  # 解包三个卷积层输出通道

        # ------------------------------
        # 主路径
        # ------------------------------
        # 第一层卷积 1x1
        self.conv1 = nn.Conv2d(
            in_channels=in_channels, 
            out_channels=F1, 
            kernel_size=1, 
            stride=s,    # 控制降采样
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(F1)

        # 第二层卷积 fxf，stride=1，padding SAME
        self.conv2 = nn.Conv2d(
            in_channels=F1, 
            out_channels=F2, 
            kernel_size=f, 
            stride=1, 
            padding=f//2, 
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(F2)

        # 第三层卷积 1x1
        self.conv3 = nn.Conv2d(
            in_channels=F2, 
            out_channels=F3, 
            kernel_size=1, 
            stride=1, 
            bias=False
        )
        self.bn3 = nn.BatchNorm2d(F3)

        # ------------------------------
        # 捷径分支 (shortcut)
        # ------------------------------
        # 卷积 1x1 用于匹配主路径输出通道和尺寸
        self.shortcut_conv = nn.Conv2d(
            in_channels=in_channels, 
            out_channels=F3, 
            kernel_size=1, 
            stride=s,   # 与主路径第一层卷积步幅相同
            bias=False
        )
        self.shortcut_bn = nn.BatchNorm2d(F3)

        # ReLU 激活
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        前向传播

        参数：
            x -- 输入 Tensor, shape = (batch_size, in_channels, H, W)

        返回：
            输出 Tensor, shape = (batch_size, F3, H_out, W_out)
        """
        # ------------------------------
        # 捷径路径
        # ------------------------------
        shortcut = self.shortcut_conv(x)
        shortcut = self.shortcut_bn(shortcut)

        # ------------------------------
        # 主路径
        # ------------------------------
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        # ------------------------------
        # 尺寸对齐（可选）
        # ------------------------------
        # 当主路径和捷径尺寸不一致时，插值对齐
        if out.shape != shortcut.shape:
            shortcut = F.interpolate(shortcut, size=out.shape[2:], mode='nearest')

        # ------------------------------
        # 残差连接 + ReLU
        # ------------------------------
        out += shortcut
        out = self.relu(out)

        return out


In [10]:
# =====================================
# 设置随机种子，保证实验可复现
# =====================================
np.random.seed(1)
torch.manual_seed(1)

# =====================================
# 构造输入张量
# =====================================
# batch_size=3, 输入通道=6, 高=4, 宽=4
X = torch.tensor(np.random.randn(3, 6, 4, 4), dtype=torch.float32)

# =====================================
# 定义卷积块
# =====================================
# 输入通道=6, 三层卷积通道=[2,4,6], 中间卷积核大小 f=2, 步幅 s=2
conv_block = ConvolutionalBlock(in_channels=6, f=2, filters=[2,4,6], s=2)

# 切换到评估模式（不计算梯度，不更新 BN 统计）
conv_block.eval()

# =====================================
# 前向传播
# =====================================
out = conv_block(X)

# =====================================
# 打印输出
# =====================================
print("输出尺寸:", out.shape)
# 展平后的前6个元素
print("out.flatten()[:6] =", out.flatten()[:6].detach().numpy())


输出尺寸: torch.Size([3, 6, 3, 3])
out.flatten()[:6] = [0.         0.         0.20515898 0.         0.         0.19972707]


## 5 - 构建你的第一个 50 层 ResNet 模型

在前面我们已经实现了 **ResNet 的恒等块（Identity Block）** 和 **卷积块（Convolutional Block）**，现在可以构建一个非常深的网络了。  
下图展示了 ResNet-50 的整体结构："ID BLOCK" 表示 Identity Block，"ID BLOCK x3" 表示堆叠 3 个恒等块。

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> 
<u> <font color='purple'> **图 5** </u><font color='purple'> ： **ResNet-50 模型** </center></caption>

---

### ResNet-50 模型详细说明

#### 输入层
- Zero-padding：对输入图像进行填充，pad=(3,3)

---

#### Stage 1
- Conv2D：
  - 卷积核数量：64
  - 卷积核大小：(7,7)
  - 步幅：(2,2)
- BatchNorm：对通道维归一化
- MaxPooling：
  - 窗口大小：(3,3)
  - 步幅：(2,2)

---

#### Stage 2
- **卷积块（Conv Block）**
  - 卷积核组大小：[64,64,256]
  - 卷积核大小 f=3
  - 步幅 s=1
  - 名称后缀："a"
- **恒等块（Identity Block）**
  - 2 个恒等块
  - 卷积核组大小：[64,64,256]
  - 卷积核大小 f=3

---

#### Stage 3
- 卷积块：
  - 卷积核组大小：[128,128,512]
  - f=3, s=2
- 恒等块：
  - 3 个恒等块
  - 卷积核组大小：[128,128,512]
  - f=3

---

#### Stage 4
- 卷积块：
  - 卷积核组大小：[256,256,1024]
  - f=3, s=2
- 恒等块：
  - 5 个恒等块
  - 卷积核组大小：[256,256,1024]
  - f=3

---

#### Stage 5
- 卷积块：
  - 卷积核组大小：[512,512,2048]
  - f=3, s=2
- 恒等块：
  - 2 个恒等块
  - 卷积核组大小：[512,512,2048]
  - f=3
---

#### 输出层
- Average Pooling：
  - 窗口大小：(2,2)
- Flatten：无超参数，无名称
- Fully Connected（Dense）层：
  - 输出维度 = 类别数
  - 激活函数：softmax

---


> 小结：  
> ResNet-50 由 5 个 stage 组成，每个 stage 由 1 个卷积块和若干恒等块组成。卷积块用于调整维度，恒等块用于学习更深的特征。最后通过平均池化和全连接层完成分类。


In [11]:
# =====================================
# ResNet50 主体定义
# =====================================
class ResNet50(nn.Module):
    """
    ResNet50 网络结构（简化版，用于小型分类任务）
    输入: (batch_size, 3, H, W)
    输出: (batch_size, num_classes)
    """
    def __init__(self, num_classes=6):
        super(ResNet50, self).__init__()

        # ------------------------------
        # Stage 1: 初始卷积 + BN + ReLU + MaxPool
        # ------------------------------
        self.pad = nn.ZeroPad2d(3)          # 四周填充 3 个像素 (left, right, top, bottom)
        self.conv1 = nn.Conv2d(
            in_channels=3,                   # 输入通道数
            out_channels=64,                 # 输出通道数
            kernel_size=7,                   # 卷积核 7x7
            stride=2,                        # 步幅 2
            bias=False                        # 不使用偏置，BN 会有偏置
        )
        self.bn1   = nn.BatchNorm2d(64)     # BatchNorm 层，规范化 64 个通道
        self.relu  = nn.ReLU(inplace=True)  # ReLU 激活
        self.maxpool = nn.MaxPool2d(
            kernel_size=3,                   # 池化窗口 3x3
            stride=2,                        # 步幅 2
            padding=1                        # SAME 填充
        )

        # ------------------------------
        # Stage 2: 1 个卷积块 + 2 个恒等块
        # ------------------------------
        self.stage2 = nn.Sequential(
            ConvolutionalBlock(64,  [64, 64, 256], f=3, s=1),
            IdentityBlock(256, [64, 64, 256], f=3),
            IdentityBlock(256, [64, 64, 256], f=3)
        )

        # ------------------------------
        # Stage 3: 1 个卷积块 + 3 个恒等块
        # ------------------------------
        self.stage3 = nn.Sequential(
            ConvolutionalBlock(256, [128, 128, 512], f=3, s=2),
            IdentityBlock(512, [128, 128, 512], f=3),
            IdentityBlock(512, [128, 128, 512], f=3),
            IdentityBlock(512, [128, 128, 512], f=3)
        )

        # ------------------------------
        # Stage 4: 1 个卷积块 + 5 个恒等块
        # ------------------------------
        self.stage4 = nn.Sequential(
            ConvolutionalBlock(512, [256, 256, 1024], f=3, s=2),
            IdentityBlock(1024, [256, 256, 1024], f=3),
            IdentityBlock(1024, [256, 256, 1024], f=3),
            IdentityBlock(1024, [256, 256, 1024], f=3),
            IdentityBlock(1024, [256, 256, 1024], f=3),
            IdentityBlock(1024, [256, 256, 1024], f=3)
        )

        # ------------------------------
        # Stage 5: 1 个卷积块 + 2 个恒等块
        # ------------------------------
        self.stage5 = nn.Sequential(
            ConvolutionalBlock(1024, [512, 512, 2048], f=3, s=2),
            IdentityBlock(2048, [512, 512, 2048], f=3),
            IdentityBlock(2048, [512, 512, 2048], f=3)
        )

        # ------------------------------
        # 全局平均池化 + 全连接输出
        # ------------------------------
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))  # 将每个通道池化成 1x1
        self.fc = nn.Linear(2048, num_classes)      # 全连接层输出类别数

    # ------------------------------
    # 前向传播
    # ------------------------------
    def forward(self, x):
        # Stage 1
        x = self.pad(x)         # 填充
        x = self.conv1(x)       # 卷积
        x = self.bn1(x)         # BN
        x = self.relu(x)        # ReLU
        x = self.maxpool(x)     # 最大池化

        # Stage 2~5
        x = self.stage2(x)      # stage2 卷积块+恒等块
        x = self.stage3(x)      # stage3
        x = self.stage4(x)      # stage4
        x = self.stage5(x)      # stage5

        # 全局池化 + 展平 + 全连接
        x = self.avgpool(x)            # 输出 (batch, 2048, 1, 1)
        x = torch.flatten(x, 1)        # 展平成 (batch, 2048)
        x = self.fc(x)                 # 全连接输出 (batch, num_classes)
        return x


运行下面的单元格，在 2 个 epoch 上训练你的模型，批量大小为 32。

In [12]:
# =====================================
# 训练和测试 ResNet50 示例
# =====================================

# ------------------------------
# 设备选择
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 如果 GPU 可用，则使用 GPU，否则使用 CPU

# ------------------------------
# 模型实例化
# ------------------------------
model = ResNet50(num_classes=6).to(device)
# 将模型移到设备（GPU/CPU）

# ------------------------------
# 损失函数和优化器
# ------------------------------
criterion = nn.CrossEntropyLoss()
# 多分类交叉熵损失
# 输入 logits (未经过 softmax) 形状 = (batch_size, num_classes)
# 标签为类别索引 (long 类型)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Adam 优化器，自适应学习率
# model.parameters() 获取所有可训练参数
# lr=0.001 为初始学习率

# ------------------------------
# 简单训练循环（2 个 epoch 示例）
# ------------------------------
for epoch in range(2):
    model.train()  # 设置模型为训练模式（启用 BN/Dropout）
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # 移动数据到设备

        outputs = model(images)                # 前向传播，得到 logits
        loss = criterion(outputs, labels)     # 计算损失

        optimizer.zero_grad()  # 梯度清零
        loss.backward()        # 反向传播计算梯度
        optimizer.step()       # 更新参数

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    # 打印当前 epoch 最后一个 batch 的损失

# ------------------------------
# 测试集评估
# ------------------------------
model.eval()  # 设置模型为评估模式（禁用 BN/Dropout）
correct, total = 0, 0

with torch.no_grad():  # 不计算梯度，节省内存
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)                     # 前向传播
        _, preds = torch.max(outputs, 1)           # 取最大值索引作为预测类别
        total += labels.size(0)                     # 累加样本总数
        correct += (preds == labels).sum().item()  # 累加正确预测数

# 打印准确率
print("准确率 = {:.2f}%".format(100 * correct / total))


Epoch 1, Loss: 0.8993
Epoch 2, Loss: 0.6568
准确率 = 36.67%


### 6 查看模型

In [13]:
model = ResNet50()  
print(model)

ResNet50(
  (pad): ZeroPad2d((3, 3, 3, 3))
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): ConvolutionalBlock(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut_conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (short

<font color='blue'>
    
**你应该记住的内容：**
- 在实践中，非常深的“普通”网络效果不佳，因为梯度容易消失，训练困难。  
- 跳跃连接（skip-connections）有助于解决梯度消失问题，同时也使 ResNet 块更容易学习恒等映射（identity function）。
- ResNet 有两种主要类型的块：恒等块（identity block）和卷积块（convolutional block）。
- 非常深的残差网络（Residual Networks）是通过堆叠这些块构建起来的。


### 参考文献

本笔记本介绍了 He 等人（2015）的 ResNet 算法。实现过程中也参考了 Francois Chollet 的 GitHub 仓库，并遵循了其结构：

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet 的 GitHub 仓库: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
